# Point Cloud Performance

> Download this data from [here](https://ugentbe-my.sharepoint.com/:f:/g/personal/victorlouis_degusseme_ugent_be/EkIZoyySsnZBg56hRq1BqdkBuGlvhAwPWT9HDuqaUB-psA?e=iSehj6) and save the folder in a folder called `data` relative to this notebook.

Main takeaways:
* "Conversion" is ~instant because array memory is shared.
* Filtering time scales with the amount of `True` values in the mask.
* Using `&` on mask is fast, so it's better to `&` lots of masks together than to filter sequentially.
* Filtering with our dataclass/in numpy is faster than Open3D's `select_by_index` or `select_by_mask`.
* Using `.nonzero()` on a boolean array before indexing is faster than using a boolean array directly.

In [30]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
import os
data_dir = os.path.join("data", "competition_sample_0000")

In [32]:
!ls $data_dir

calibration	  confidence_map.tiff  image_left.png	intrinsics.json
camera_pose.json  depth_map.tiff       image_right.png	pointcloud.ply


In [33]:
import cv2
import numpy as np
import open3d as o3d

from airo_typing import NumpyDepthMapType
from airo_dataset_tools.data_parsers.camera_intrinsics import CameraIntrinsics
from airo_dataset_tools.data_parsers.pose import Pose
from airo_camera_toolkit.point_clouds.conversions import open3d_to_point_cloud, point_cloud_to_open3d
from airo_camera_toolkit.point_clouds.operations import filter_point_cloud, crop_point_cloud
from airo_camera_toolkit.point_clouds.operations import generate_point_cloud_crop_mask


In [34]:
intrinsics_path = os.path.join(data_dir, "intrinsics.json")
image_left_path = os.path.join(data_dir, "image_left.png")
image_right_path = os.path.join(data_dir, "image_right.png")
depth_map_path = os.path.join(data_dir, "depth_map.tiff")
confidence_map_path = os.path.join(data_dir, "confidence_map.tiff")
point_cloud_path = os.path.join(data_dir, "pointcloud.ply")
camera_pose_path = os.path.join(data_dir, "camera_pose.json")

In [35]:
with open(intrinsics_path, "r") as f:
    intrinsics_model = CameraIntrinsics.model_validate_json(f.read())
    intrinsics = intrinsics_model.as_matrix()
    resolution = intrinsics_model.image_resolution.as_tuple()

with open(camera_pose_path, "r") as f:
    camera_pose = Pose.model_validate_json(f.read()).as_homogeneous_matrix()

with np.printoptions(precision=3, suppress=True):
    print("Resolution:", resolution)
    print("Intrinsics: \n", intrinsics)
    print("Extrinsics: \n", camera_pose)

Resolution: (2208, 1242)
Intrinsics: 
 [[1056.831    0.    1103.95 ]
 [   0.    1056.831  621.699]
 [   0.       0.       1.   ]]
Extrinsics: 
 [[ 1.    -0.001 -0.004  0.475]
 [ 0.003 -0.365  0.931 -1.318]
 [-0.002 -0.931 -0.365  0.9  ]
 [ 0.     0.     0.     1.   ]]


In [36]:
def binarize_confidence(image: NumpyDepthMapType, threshold=50.0):
    confident = image <= threshold
    return confident

depth_map = cv2.imread(depth_map_path, cv2.IMREAD_ANYDEPTH)
confidence_map = cv2.imread(confidence_map_path, cv2.IMREAD_ANYDEPTH)

# Confidence mask
threshold = 1.0  # a value of 1.0 means only the most confidence points will be kept
confidence_binarized = binarize_confidence(confidence_map, threshold)
confidence_mask = confidence_binarized.reshape(-1)

# Bounding box
bbox = (0.35, -0.3, 0.1), (0.7, 0.1, 0.95)
bbox_o3d = o3d.t.geometry.AxisAlignedBoundingBox(*bbox)

# Open3D point clouds
pcd_in_camera = o3d.t.io.read_point_cloud(point_cloud_path)

# This conversion to float32 can be removed once the data is saved as float32
pcd_in_camera.point.positions = o3d.core.Tensor(pcd_in_camera.point.positions.numpy().astype(np.float32))

pcd = pcd_in_camera.transform(camera_pose) # transform to world frame (= base frame of left robot)
pcd_filtered = pcd.select_by_mask(confidence_mask)
pcd_cropped = pcd_filtered.crop(bbox_o3d)

# Airo-mono point clouds
point_cloud = open3d_to_point_cloud(pcd)
point_cloud_filtered = filter_point_cloud(point_cloud, confidence_mask)
point_cloud_cropped = crop_point_cloud(point_cloud_filtered, bbox)

pcd

PointCloud on CPU:0 [2742336 points (Float32)].
Attributes: colors (dtype = UInt8, shape = {2742336, 3}).

In [37]:
print(f"Confidence mask: {100.0 * confidence_mask.mean():.2f}% is True")

Confidence mask: 85.61% is True


### Performance comparison

#### Conversion

In [38]:
%%timeit
point_cloud_to_open3d(point_cloud)

13.7 µs ± 320 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [39]:
%%timeit
open3d_to_point_cloud(pcd)

17.3 µs ± 54.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


#### Filtering

In [40]:
%%timeit
filter_point_cloud(point_cloud, confidence_mask.nonzero()) # adding nonzero() is faster

59.3 ms ± 160 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [41]:
%%timeit
filter_point_cloud(point_cloud, confidence_mask)

67.5 ms ± 204 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [42]:
np.all(point_cloud.points[confidence_mask] == point_cloud.points[confidence_mask.nonzero()])

True

In [43]:
confidence_mask_95_false = confidence_mask.copy()
confidence_mask_95_false[:int(0.95 * len(confidence_mask))] = False

In [44]:
%%timeit
filter_point_cloud(point_cloud, confidence_mask_95_false.nonzero())

2.95 ms ± 7.01 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [45]:
%%timeit
pcd.select_by_index(np.where(confidence_mask)[0])

140 ms ± 608 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [46]:
%%timeit
pcd.select_by_mask(confidence_mask)

238 ms ± 3.37 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [47]:
pcd_legacy = pcd.to_legacy()

In [48]:
%%timeit
pcd_legacy.select_by_index(np.where(confidence_mask)[0])

388 ms ± 3.16 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Cropping

In [49]:
%%timeit
crop_point_cloud(point_cloud_filtered, bbox)

16.9 ms ± 135 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [50]:
%%timeit
pcd_filtered.crop(bbox_o3d)

31.1 ms ± 903 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### Downsampling

In [51]:
%%timeit
pcd_cropped.voxel_down_sample(voxel_size=0.01)

3.04 ms ± 37.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### Removing low confidence points and cropping as fast as possible

In [52]:
crop_mask = generate_point_cloud_crop_mask(point_cloud, bbox)

In [53]:
%%timeit
crop_mask & confidence_mask

139 µs ± 3.25 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [54]:
%%timeit
crop_mask = generate_point_cloud_crop_mask(point_cloud, bbox)
mask = crop_mask & confidence_mask
filter_point_cloud(point_cloud, mask.nonzero())

21.1 ms ± 252 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [55]:
%%timeit
point_cloud_filtered = filter_point_cloud(point_cloud, confidence_mask.nonzero())
point_cloud_cropped = crop_point_cloud(point_cloud_filtered, bbox)

74.9 ms ± 231 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [56]:
%%timeit
crop_mask = generate_point_cloud_crop_mask(point_cloud, bbox)
mask = crop_mask & confidence_mask
pcd.select_by_index(mask.nonzero())

25 ms ± 102 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [57]:
pcd2 = o3d.t.geometry.PointCloud(pcd)
pcd2.point.confidence = o3d.core.Tensor(confidence_mask)

In [58]:
%%timeit
pcd2_cropped = pcd2.crop(bbox_o3d)
pcd2_cropped.select_by_index(pcd2_cropped.point.confidence.numpy().nonzero())

49.8 ms ± 160 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
